# Vault Transfers

*Work in Progress*

This notebook aims to provide a brief demonstration of how this codebase could be used for visualize vault transfers.

In [1]:
"""load dependencies and define constants"""
import os
import sys
import numpy as np
import pandas as pd
from decimal import Decimal
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from typing import Literal

sys.path.append("..")
from src.yearn import Network, Yearn, Subgraph, Vault
from src.utils.web3 import Web3Provider 
from src.utils.network import client, parse_json

# constants
ZERO_ADDRESS = '0x0000000000000000000000000000000000000000'
FROM_BLOCK = 13900000  # Dec-29-2021
BATCH_SIZE = 500000
API_ENDPOINT = "https://api.etherscan.io/api"

pd.set_option("expand_frame_repr", False)

In [2]:
"""get current block and load vaults from the yearn instance"""
load_dotenv()
w3 = Web3Provider(Network.Mainnet)
current_block = w3.provider.eth.get_block("latest")["number"]
print(f"current block number: {current_block}")

yearn = Yearn(Network.Mainnet)
vaults = yearn.vaults
print(f"loaded metadata for {len(vaults)} vaults (v2)")

subgraph = Subgraph(Network.Mainnet)

current block number: 14953131
loaded metadata for 101 vaults (v2)


In [4]:
"""get addresses that withdrew from vaults"""
withdrawals_stats = {"vaults": {}, "overall_account_transfers": {} }
stat_vals = ["count", "shares", "shares_usdc"]
for idx, vault in enumerate(vaults):
    # approximate the share price by the current usdc value, skip if not available
    try:
        share_price = w3.get_usdc_price(vault.token.address)
    except:
        continue

    transfers = subgraph.transfers("withdrawals", vault, from_block=FROM_BLOCK, to_block=current_block)

    # vault's count and shares stats
    _dict = {
        "count": transfers.count,
        "shares": transfers.shares,
        "shares_usdc": float(Decimal(transfers.shares) * share_price),
        "account_transfers": transfers.account_transfers
    }
    print(f"vault: {vault.address} {dict((k, v) for k, v in _dict.items() if k in stat_vals)}")

    # add vault to withdrawals stats
    withdrawals_stats["vaults"][vault.address] = _dict

    account_items = transfers.account_transfers.items()
    account_transfers = sorted(account_items, key=lambda item: item[1].shares, reverse=True)
    
    for index, (account, transfer) in enumerate(account_transfers[:10]):
        # vault's account stats
        _dict = {
            "count": transfer.count,
            "shares": transfer.shares,
            "shares_usdc": float(Decimal(transfer.shares) * share_price)
        }
        print(f"{index}: {account} {_dict}")

        # add account_transfers to withdrawals stats
        overall_account_transfers = withdrawals_stats["overall_account_transfers"]
        if overall_account_transfers.get(account) is None:
            overall_account_transfers[account] = {}
        account_stats = overall_account_transfers[account]
        overall_account_transfers[account] = {k: account_stats.get(k, 0) + _dict[k] for k in _dict}
    print()

# withdrawals stats for vaults and account_transfers
for key in withdrawals_stats.keys():
    vault_stats = []
    for subkey, subval in withdrawals_stats[key].items():
        # only include items in stat_vals, plus address
        new_val = dict((k, v) for k, v in subval.items() if k in stat_vals)
        vault_stats.append((subkey, *new_val.values()))
    df = pd.DataFrame(vault_stats, columns=["address", "count", "shares", "approx. USDC value"])
    df["approx. USDC value"] = pd.to_numeric(df["approx. USDC value"])
    df.set_index(keys=["address"], inplace=True)
    df.sort_values("approx. USDC value", inplace=True, ascending=False)
    print(f"top {key}")
    print(df.head(10))
    print()

vault: 0xf8768814b88281DE4F532a3beEfA5b85B69b9324 {'count': 0, 'shares': 0, 'shares_usdc': 0.0}

vault: 0xF29AE508698bDeF169B89834F76704C3B205aedf {'count': 473, 'shares': 743867.3464392548, 'shares_usdc': 1653138.0605633566}
0: 0xee91ae665be6a92a1241911cea3cae9d50fe658d {'count': 159, 'shares': 174095.01569802006, 'shares_usdc': 386901.10270658904}
1: 0xf929122994e177079c924631ba13fb280f5cd1f9 {'count': 1, 'shares': 67609.53234890274, 'shares_usdc': 150252.4498727781}
2: 0x597568694fe6ee1b701ec8578bd57102c9a29abd {'count': 1, 'shares': 48630.865059603995, 'shares_usdc': 108075.0947504013}
3: 0x0dac1e97d866cad71fd9c71abb904612443fc755 {'count': 1, 'shares': 33342.18245935315, 'shares_usdc': 74098.19924163823}
4: 0x3bbacdd15eaa989a68cdbf0dd81c7c1f58e9fa6e {'count': 1, 'shares': 32783.11190934428, 'shares_usdc': 72855.74545040273}
5: 0x82f0d518c6a7735dfe2371cf0c25b9a80d9d554b {'count': 1, 'shares': 25771.081864828862, 'shares_usdc': 57272.518408793614}
6: 0xd6b88257e91e4e4d4e990b3a858c84

In [6]:
"""get addresses that deposited to vaults"""
deposits_stats = { "vaults": {}, "overall_account_transfers": {} }
stat_vals = ["count", "shares", "shares_usdc"]
for idx, vault in enumerate(vaults):
    # approximate the share price by the current usdc value, skip if not available
    try:
        share_price = w3.get_usdc_price(vault.token.address)
    except:
        continue
    
    transfers = subgraph.transfers("deposits", vault, from_block=FROM_BLOCK, to_block=current_block)

    # vault's count and shares stats
    _dict = {
        "count": transfers.count,
        "shares": transfers.shares,
        "shares_usdc": float(Decimal(transfers.shares) * share_price),
        "account_transfers": transfers.account_transfers
    }
    print(f"vault: {vault.address} {dict((k, v) for k, v in _dict.items() if k in stat_vals)}")

    # add vault to deposits stats
    deposits_stats["vaults"][vault.address] = _dict

    account_items = transfers.account_transfers.items()
    account_transfers = sorted(account_items, key=lambda item: item[1].shares, reverse=True)
    
    for index, (account, transfer) in enumerate(account_transfers[:10]):
        # vault's account stats
        _dict = {
            "count": transfer.count,
            "shares": transfer.shares,
            "shares_usdc": float(Decimal(transfer.shares) * share_price)
        }
        print(f"{index}: {account} {_dict}")

        # add account_transfers to deposits stats
        overall_account_transfers = deposits_stats["overall_account_transfers"]
        if overall_account_transfers.get(account) is None:
            overall_account_transfers[account] = {}
        account_stats = overall_account_transfers[account]
        overall_account_transfers[account] = {k: account_stats.get(k, 0) + _dict[k] for k in _dict}
    print()

# deposits stats for vaults and account_transfers
for key in deposits_stats.keys():
    vault_stats = []
    for subkey, subval in deposits_stats[key].items():
        # only include items in stat_vals, plus address
        new_val = dict((k, v) for k, v in subval.items() if k in stat_vals)
        vault_stats.append((subkey, *new_val.values()))
    df = pd.DataFrame(vault_stats, columns=["address", "count", "shares", "approx. USDC value"])
    df["approx. USDC value"] = pd.to_numeric(df["approx. USDC value"])
    df.set_index(keys=["address"], inplace=True)
    df.sort_values("approx. USDC value", inplace=True, ascending=False)
    print(f"top {key}")
    print(df.head(10))
    print()

vault: 0xf8768814b88281DE4F532a3beEfA5b85B69b9324 {'count': 0, 'shares': 0, 'shares_usdc': 0.0}

vault: 0xF29AE508698bDeF169B89834F76704C3B205aedf {'count': 242, 'shares': 156623.11737201488, 'shares_usdc': 344219.3959430499}
0: 0xee91ae665be6a92a1241911cea3cae9d50fe658d {'count': 154, 'shares': 59126.61994599614, 'shares_usdc': 129945.8837460327}
1: 0x82f0d518c6a7735dfe2371cf0c25b9a80d9d554b {'count': 1, 'shares': 19303.86531563336, 'shares_usdc': 42425.18582062511}
2: 0x5201103dfc394b7a510e7e1fb3c9159f414cca12 {'count': 1, 'shares': 16393.913809398724, 'shares_usdc': 36029.8224380889}
3: 0x595f16d20959880bfede2c95adb86762f5d155da {'count': 1, 'shares': 8259.593261022846, 'shares_usdc': 18152.570646972526}
4: 0x08e5e4df2f56be734ead5c80a3b2377616210a62 {'count': 2, 'shares': 7489.342796064911, 'shares_usdc': 16459.748066108437}
5: 0x995a09ed0b24ee13fbfcfbe60cad2fb6281b479f {'count': 1, 'shares': 7221.89181410995, 'shares_usdc': 15871.956065811028}
6: 0xf919e7804dc4c413f5c3f523f293b60e4

In [18]:
def equalize_lists(*args: list[tuple]):
    # Make all lists equal in size
    items = [*args]
    max_length = 0
    for item in items:
        max_length = max(max_length, len(item))
    for item in items:
        item += [(None, None)] * (max_length - len(item))

"""get net transfers of addresses to vaults"""
def get_net_transfers(dict_key: str) -> dict[str, dict[str, float]]:
    net_transfers = {}
    withdrawals_items = list(withdrawals_stats[dict_key].items())
    deposits_items = list(deposits_stats[dict_key].items())
    equalize_lists(withdrawals_items, deposits_items)

    keys = ["shares", "shares_usdc"]
    for (withdraw_address, withdraw_stats), (deposit_address, deposit_stats) in zip(withdrawals_items, deposits_items):
        if withdraw_address is not None:
            if net_transfers.get(withdraw_address) is None:
                net_transfers[withdraw_address] = {}
            net_transfers[withdraw_address] = {k: net_transfers[withdraw_address].get(k, 0) - withdraw_stats[k] for k in keys}
        if deposit_address is not None:
            if net_transfers.get(deposit_address) is None:
                net_transfers[deposit_address] = {}
            net_transfers[deposit_address] = {k: net_transfers[deposit_address].get(k, 0) + deposit_stats[k] for k in keys}

    return net_transfers

In [24]:
vaults = get_net_transfers("vaults")
overall_account_transfers = get_net_transfers("overall_account_transfers")

In [ ]:
# net_transfers = {}
# withdrawals_items = withdrawals_stats["vaults"].items()
# deposits_items = deposits_stats["vaults"].items()